In [1]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-dltm8stp
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-dltm8stp
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=195b08e186a5b2573a2384ddf5fff872479a23e69f2c786da5af2558b6df7923
  Stored in directory: /tmp/pip-ephem-wheel-cache-quosohie/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


#### Question 1

In [8]:
%%cu
#include <stdio.h>
#include <stdlib.h>
__managed__ int sum=0;
__global__ void add(int *arr,int *n)
{
    int th = threadIdx.x;
    if(th < *n)
      atomicAdd(&sum,arr[th]);
}

int main()
{
    int n=8;
    int arr[n];
    int *d_arr,*d_n;
 
    for( int i=0;i<n;i++)
    {
        arr[i]=rand()%100;
        printf("%d ",arr[i]);
    }
    printf("\n");

    cudaMalloc((void**)&d_arr,n*sizeof(int));
    cudaMalloc((void**)&d_n,sizeof(int));
    
    cudaMemcpy(d_arr,arr,n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(d_n,&n,sizeof(int),cudaMemcpyHostToDevice);
    add <<<1,n>>>(d_arr,d_n);
    printf("Sum:%d\n",sum);
    cudaFree(d_arr);
    cudaFree(d_n);
    return 0;
}

83 86 77 15 93 35 86 92 
Sum:567



#### Question 2

In [12]:
%%cu
#include <stdio.h>
#include <stdlib.h>

__global__ void add(int *a,int *b,int *c,int *n)
{
    unsigned short th =threadIdx.x;
    if(th < *n)
      c[th] = a[th]+b[th];
}

int main()
{
    int n=8;
    int a[n],b[n],c[n];
    int *d_a,*d_b,*d_c,*d_n;
 
    for(int i=0;i<n;i++)
      a[i]=rand()%100;
    for(int i=0;i<n;i++)
      b[i]=rand()%100;

    cudaMalloc((void**)&d_a,n*sizeof(int));
    cudaMalloc((void**)&d_b,n*sizeof(int));
    cudaMalloc((void**)&d_c,n*sizeof(int));
    cudaMalloc((void**)&d_n,sizeof(int));
    
    cudaMemcpy(d_a,a,n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(d_b,b,n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(d_n,&n,sizeof(int),cudaMemcpyHostToDevice);
 
    add <<<1,n>>>(d_a,d_b,d_c,d_n);
    cudaMemcpy(c,d_c,n*sizeof(int),cudaMemcpyDeviceToHost);
 
    for( int i=0;i<n;i++)
    printf("%d + %d = %d\n",a[i],b[i],c[i]);
 
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    cudaFree(d_n);
    return 0;
}

83 + 49 = 132
86 + 21 = 107
77 + 62 = 139
15 + 27 = 42
93 + 90 = 183
35 + 59 = 94
86 + 63 = 149
92 + 26 = 118



#### Question 3

In [17]:
%%cu
#include <stdio.h>
#include <stdlib.h>

__global__ void mult(int *a,int *b,int *c,int *m,int *n,int *p)
{
    int row=blockIdx.x*blockDim.x+threadIdx.x;
    int col=blockIdx.y*blockDim.y+threadIdx.y;
    
    int t=0;
    if(row<*m && col<*p)
    for(int i=0;i<*n;i++)
      t= t + a[row*(*n)+i] * b[i*(*p)+col];
    c[row*(*p)+col] = t;
}

int main()
{
    int i,j;
    int m=2,n=2,p=2;

    int a[m*n],b[n*p],c[m*n];
    int *d_a,*d_b,*d_c,*d_m,*d_n,*d_p;
    printf("Matrix A:\n");
    for(i=0;i<m;i++)
    {
        for(j=0;j<n;j++)
        {
            a[i*n+j]=rand()%100;
            printf("%d ",a[i*n+j]);       
        }
        printf("\n");
    }
    printf("Matrix B:\n");
    for(i=0;i<n;i++)
    {
        for(j=0;j<p;j++)
        {
            b[i*p+j]=rand()%100;
            printf("%d ",b[i*p+j]);       
        }
        printf("\n");
    }
    printf("\n");

    cudaMalloc((void**)&d_a,m*n*sizeof(int));
    cudaMalloc((void**)&d_b,n*p*sizeof(int));
    cudaMalloc((void**)&d_c,m*p*sizeof(int));
    cudaMalloc((void**)&d_m,sizeof(int));
    cudaMalloc((void**)&d_n,sizeof(int));
    cudaMalloc((void**)&d_p,sizeof(int));

    cudaMemcpy(d_a,a,m*n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(d_b,b,n*p*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(d_m,&m,sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(d_n,&n,sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(d_p,&p,sizeof(int),cudaMemcpyHostToDevice);
 
    dim3 threadsPerBlock(m,p);
    dim3 blocksPerGrid(1,1);
        

    mult<<<blocksPerGrid,threadsPerBlock>>> (d_a,d_b,d_c,d_m,d_n,d_p);   

    cudaMemcpy(c,d_c,m*p*sizeof(int),cudaMemcpyDeviceToHost);
    printf("multiply two matrices:  \n");
    for(i=0;i<m;i++)
    {
        for(j=0;j<p;j++)
        {
            printf("%d ",c[i*p+j]);       
        }
        printf("\n");
    }
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    cudaFree(d_m);
    cudaFree(d_n);
    cudaFree(d_p);
    return 0;
}

Matrix A:
83 86 
77 15 
Matrix B:
93 35 
86 92 

multiply two matrices:  
15115 10817 
8451 4075 

